# Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Loading the dataset

In [2]:
filepath = ".\\KPMG_VI_New_raw_data_update_final.xlsx"

## Transactions Sheet

In [6]:
def load_sheet(sheet_name):
    """
    This function uses pandas,
    to read an excel sheet from a workbook,
    and returns a pandas dataframe
    """
    df = pd.read_excel(filepath, sheet_name=sheet_name, header=1)
    return df

In [7]:
tran_df = load_sheet("Transactions")
tran_df.head()

transaction_id  product_id  customer_id transaction_date  online_order  \
0               1           2         2950       2017-02-25           0.0   
1               2           3         3120       2017-05-21           1.0   
2               3          37          402       2017-10-16           0.0   
3               4          88         3135       2017-08-31           0.0   
4               5          78          787       2017-10-01           1.0   

  order_status           brand product_line product_class product_size  \
0     Approved           Solex     Standard        medium       medium   
1     Approved   Trek Bicycles     Standard        medium        large   
2     Approved      OHM Cycles     Standard           low       medium   
3     Approved  Norco Bicycles     Standard        medium       medium   
4     Approved  Giant Bicycles     Standard        medium        large   

   list_price  standard_cost  product_first_sold_date  
0       71.49          53.62                  41245.0  
1     2091.47         388.92                  41701.0  
2     1793.43         248.82                  36361.0  
3     1198.46         381.10                  36145.0  
4     1765.30         709.48                  42226.0

In [5]:
tran_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

There are a couple of issues with this dataframe;
1. Missing values in columns:

* online_order,
* brand,
* product_line,
* product_class,
* product_size,  
* standard_cost, and
* product_first_sold_date

2. Incorrect datatype for *product_first_sold_date* column

### Exploratory Data Analysis

In [8]:
tran_df['product_id'].nunique()

101

In [9]:
tran_df['customer_id'].nunique()

3494

In [11]:
tran_df['transaction_id'].max()

20000

We have 101 unique products sold to 3494 unique customers over 20,000 transactions, in this transactions sheet.

In [12]:
#converting serial date format to datetime
import xlrd

def convert_date(serial_date):
    """
    This function takes in a date in serial format,
    and converts it into pandas datetime format,
    using xlrd library
    """
    date_time_date = xlrd.xldate_as_datetime(serial_date, 0)
    date_object = date_time_date.date()
    return date_object

In [14]:
tran_df['product_first_sold_date'].describe()

count    19803.000000
mean     38199.776549
std       2875.201110
min      33259.000000
25%      35667.000000
50%      38216.000000
75%      40672.000000
max      42710.000000
Name: product_first_sold_date, dtype: float64

In [15]:
#filling in missing date values with mode
tran_df['product_first_sold_date'].fillna(tran_df['product_first_sold_date'].mode()[0], inplace=True)

In [17]:
#using convert_date function to convert the dates in product_first_sold_date column
tran_df['product_first_sold_date'] = tran_df['product_first_sold_date'].apply(convert_date)

In [18]:
tran_df['product_first_sold_date'].head()

0    2012-12-02
1    2014-03-03
2    1999-07-20
3    1998-12-16
4    2015-08-10
Name: product_first_sold_date, dtype: object

#### Question
Which is the most sold product line?

In [21]:
product_line_sales = tran_df['transaction_id'].groupby(tran_df['product_line']).count()
product_line_sales.sort_values(ascending=False)

product_line
Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: transaction_id, dtype: int64

The **Standard** product line is the most sold, whilst the **Mountain** is the least sold.

#### Question
Which brand is the most valueable ?

In [23]:
average_brand_value = tran_df['list_price'].groupby(tran_df['brand']).mean()
average_brand_value.sort_values(ascending=False)

brand
WeareA2B          1263.454813
Giant Bicycles    1235.407204
Trek Bicycles     1183.783398
Solex             1057.271956
OHM Cycles         983.706983
Norco Bicycles     913.202450
Name: list_price, dtype: float64

The **WeareA2B** brand is the most valueable, and the **Norco Bicycles** the least.